## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate
        
dom = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
def sobel_filters(g_blur):
    S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    S2 = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Gx = cv2.filter2D(g_blur, -1, S1)
    Gy = cv2.filter2D(g_blur, -1, S2)
    
    magnitude = np.sqrt(Gx**2 + Gy**2)
    alpha = np.arctan2(Gy, Gx)
    
    return magnitude, alpha

In [ ]:
def quantization(img, alpha):
    M, N = img.shape
    quant_matrix = np.zeros((M,N), dtype=np.int32)
    
    in_degrees = alpha * 180 / np.pi
    in_degrees[in_degrees < 0] += 180

    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
               # alpha = 0 (degrees)
                if (0 <= in_degrees[i,j] < 22.5) or (157.5 <= in_degrees[i,j] <= 180):
                    quant_matrix[i,j] = 1
                    
                # alpha = 45 (degrees)
                elif (22.5 <= in_degrees[i,j] < 67.5):
                    quant_matrix[i,j] = 2
                    
                # alpha = 90 (degrees)
                elif (67.5 <= in_degrees[i,j] < 112.5):
                    quant_matrix[i,j] = 3
                    
                # alpha = 135 (degrees)
                elif (112.5 <= in_degrees[i,j] < 157.5):
                    quant_matrix[i,j] = 4

            except IndexError as e:
                pass
    
    return quant_matrix

In [ ]:
def nonmax(Z, mgtd):   # Z is matrix of directions, mgtd is magnitude
    X, Y = Z.shape
    g_N = np.zeros((X, Y))
    
    for i in range (1, X-1):
        for j in range (1, Y-1):
            if(Z[i,j] == 1):
                if(mgtd[i,j-1] > mgtd[i,j] or mgtd[i,j+1] > mgtd[i,j]):
                    pass
                else:
                    g_N[i,j] = mgtd[i,j]
                    
            elif(Z[i,j] == 2):
                if(mgtd[i+1,j-1] > mgtd[i,j] or mgtd[i-1,j+1] > mgtd[i,j]):
                    pass
                else:
                    g_N[i,j] = mgtd[i,j]
                    
            elif(Z[i,j] == 3):
                if(mgtd[i+1,j] > mgtd[i,j] or mgtd[i-1,j] > mgtd[i,j]):
                    pass
                else:
                    g_N[i,j] = mgtd[i,j]
                    
            elif(Z[i,j] == 4):
                if(mgtd[i-1,j-1] > mgtd[i,j] or mgtd[i+1,j+1] > mgtd[i,j]):
                    pass
                else:
                    g_N[i,j] = mgtd[i,j]
                    
    return g_N

In [ ]:
def complete_canny(img, weak, strong):
    g_blur = cv2.GaussianBlur(img, (3, 3), 0)
    
    magnitude, alpha = sobel_filters(g_blur)
    
    Z = quantization(img, alpha)
    
    g_N = nonmax(Z, magnitude)
    
    g_NH = g_N >= strong
    
    AND = np.logical_and(weak <= g_N, g_N < strong)
    g_NL = np.where(AND, 1, 0)
    
    
    # ==== COMPLETING CANNY ====
    
    size = img.shape
    final_result = np.zeros(size)
    
    queue = []
    for row in range(size[0]):
        for col in range(size[1]):
            if g_NH[row][col] > 0:
                queue.append([row, col])
    
    visited = np.zeros(size)
    
    while queue:
        coord = queue.pop()
        visited[coord[0]][coord[1]] = 1
        final_result[coord[0]][coord[1]] = 1

        for x in range(coord[0] - 1, coord[0] + 2):
            if 0 <= x < size[0]:
                for y in range(coord[1] - 1, coord[1] + 2):
                    if 0 <= y < size[1]:
                        if not visited[x, y] and g_NL[x][y]:
                            queue.append([x, y])
    
    return final_result, g_NH

In [ ]:
result, g_NH = complete_canny(dom, 5, 10)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 16)) 
ax[0].imshow(dom, 'gray')
ax[0].axis('off')
ax[0].set_title('Obraz wejściowy')
ax[1].imshow(g_NH, 'gray')
ax[1].axis('off')
ax[1].set_title('Pewne krawędzie')
ax[2].imshow(result, 'gray')
ax[2].axis('off')
ax[2].set_title('Obraz wynikowy')
plt.show()

In [ ]:
my_result, g_NH = complete_canny(dom, 5, 15)
cv_result = cv2.Canny(dom, 5, 15, None, 3, 1)

fig, ax = plt.subplots(1, 2, figsize=(16, 16)) 
ax[0].imshow(my_result, 'gray')
ax[0].axis('off')
ax[0].set_title('Wynik działania zaimplementowanej funkcji')
ax[1].imshow(cv_result, 'gray')
ax[1].axis('off')
ax[1].set_title('Wynik działania funkcji z biblioteki')
plt.show()